# Cell Tower Coverage

## Objective and Prerequisites

Want to learn how to configure a network of cell towers to provide signal coverage to the largest number of people possible? In this example, you’ll learn how to solve this simple covering problem. We’ll show you how to construct a mixed-integer programming (MIP) model of the problem, implement this model in the Gurobi Python API, and find an optimal solution using the Gurobi Optimizer.

This modeling example is at the beginner level, where we assume that you know Python and that you have some knowledge about building mathematical optimization models.

**Download the Repository:** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

## Motivation

Over the last ten years, smartphones have revolutionized our lives in ways that go well beyond how we communicate. Besides calling, texting, and emailing, more than two billion people around the world now use these devices to navigate to book cab rides, to compare product reviews and prices, to follow the news, to watch movies, to listen to music, to play video games,to take photographs, to participate in social media, and for numerous other applications.

A cellular network is a network of handheld smartphones in which each phone communicates with the telephone network by radio waves through a local antenna at a cellular base station (cell tower). One important problem is the placement of cell towers to provide signal coverage to the largest number of people.

## Problem Description

A telecom company needs to build a set of cell towers to provide signal coverage  for the inhabitants of a given city. A number of potential locations where the towers could be built have been identified. The towers have a fixed range, and -due to budget constraints- only a limited number of them can be built. Given these restrictions, the company wishes to provide coverage to the largest percentage of the population possible. To simplify the problem, the company has split the area it wishes to cover into a set of regions, each of which has a known population. The goal is then to choose  which of the potential locations the company should build cell towers on -in order to provide coverage to as many people as possible.

The Cell Tower Coverage Problem is an instance of the Maximal Covering Location Problem [1]. It is also related to the Set Cover Problem. Set covering problems occur in many different fields, and very important applications come from the airlines industry. For example, Crew Scheduling and Tail Assignment Problem [2].

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a mixed-integer programming (MIP) formulation for the Cell Tower Coverage Problem.

## Model Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in R$: Index and set of regions.

$G(T,R,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower,  the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in R$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at region $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if region $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function(s)

- **Population covered**. We seek to maximize the total population covered by the towers.

\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### Constraints

- **Coverage**. For each region $j \in R$ ensure that at least one tower that covers a region must be selected.

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## Python Implementation

This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| Tower 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| Tower 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| Tower 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| Tower 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| Tower 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| Tower 5 | - | - | - | 1 | 1 | - | - | - |  1 |

The population at each region is stated in the following table.

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Population | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

The cost to build a cell tower at each location site is stated inthe following table.

| <i></i> | Cost (millions of USD) |
| --- | --- |
| Tower 0 | 4.2 |
| Tower 1 | 6.1 |
| Tower 2 | 5.2 |
| Tower 3 | 5.5 |
| Tower 4 | 4.8 |
| Tower 5 | 9.2 | 

The allocated budget is $\$20,000,000$.

We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.


In [ ]:
%pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

# Parameters
budget = 20
regions, population = gp.multidict({

    1: 28459, 2: 105988, 3: 53252, 4: 101085, 5: 65079, 6: 45741, 7: 70122,
    8: 44629, 9: 370123, 10: 28820, 11: 39824, 12: 137811, 13: 198859,
    14: 41913, 15: 107419, 16: 36931, 17: 43310, 18: 1270331, 19: 52653,
    20: 39005, 21: 178535, 22: 76687, 23: 147174, 24: 28867, 25: 1180800,
    26: 42363, 27: 31031, 28: 93292, 29: 163566, 30: 39910, 31: 800658,
    32: 74912, 33: 31806, 34: 15790, 35: 28002, 36: 43378, 37: 29466,
    38: 42793, 39: 59416, 40: 33124, 41: 69035, 42: 61283, 43: 229988,
    44: 62415, 45: 167164, 46: 45623, 47: 301874, 48: 439359, 49: 43114,
    50: 237363, 51: 66599, 52: 173504, 53: 23648, 54: 40715, 55: 102734,
    56: 14613, 57: 534558, 58: 15036, 59: 34810, 60: 86243, 61: 14752,
    62: 41320, 63: 19369, 64: 36232, 65: 55948, 66: 28479, 67: 161875,
    68: 41986, 69: 34385, 70: 123144, 71: 77581, 72: 60609, 73: 79408,
    74: 56522, 75: 49259, 76: 374511, 77: 541325, 78: 208834, 79: 92483,
    80: 53095, 81: 28625, 82: 13380, 83: 215332, 84: 61607, 85: 114649,
    86: 37584, 87: 127124, 88: 22643

})

sites, coverage, cost = gp.multidict({
    1: [{1,2,18,19}, 1],
    2: [{1,2,3,17,18}, 1],
    3: [{2,3,4,17}, 1],
    4: [{3,4,5,15,16,17}, 1],
    5: [{0,2,6,7,8}, 4.8],
    6: [{3,4,8}, 9.2]
})


### Model Deployment

We now determine the model for the Cell Tower Coverage Problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to build towers that maximizes  the coverage of the population given the budget allocated.

In [ ]:
# MIP  model formulation
m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize() 

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xfa0fabb2
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 7.051000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7051.0000000 7051.00000  0.00%     -    0s

Explored 0 nodes (1 simplex i

## Analysis
The result of the optimization model shows that the maximum population that can be covered with the $\$20,000,000$ budget is 7,051 people. Let's see the solution that achieves that optimal result.

### Cell Tower Build Plan

This plan determines at which site locations to build a cell tower.

In [ ]:
# display optimal values of decision variables

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower}.")


 Build a cell tower at location Tower 0.

 Build a cell tower at location Tower 2.

 Build a cell tower at location Tower 4.


### Demand Fulfillment Metrics

- **Coverage**: Percentage of the population covered by the cell towers built.

In [ ]:
# Percentage of the population covered by the cell towers built is computed as follows.

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")


 The population coverage associated to the cell towers build plan is: 100.0 %


### Resources Utilization Metrics

- **Budget consumption**: Percentage of the budget allocated to build the cell towers.

In [ ]:
# Percentage of budget consumed to build cell towers

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The percentage of budget consumed associated to the cell towers build plan is: 71.0 %


##  Conclusion

In this example, we address the problem of building cell towers to provide signal coverage  to the largest number of people while satisfying a budget constraint. We learned how to formulate the problem as a MIP model. Also, we learned how to implement the MIP model formulation and solve it using the Gurobi Python API.

## References

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/

Copyright © 2020 Gurobi Optimization, LLC